In [1]:
import numpy as np
import cv2
import operator
import os
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import datetime
import time
import queue
from multiprocessing import Queue
from PIL import Image
import xlsxwriter

Using TensorFlow backend.


In [2]:
# module level variables
MIN_CONTOUR_AREA = 100

RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

In [3]:
#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model.h5'
# loading up our datasets
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'
 
# number of epochs to train top model 
epochs = 7 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 50 

In [ ]:
#Loading vgc16 model
vgg16 = applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255) 
#needed to create the bottleneck .npy files

W0912 13:39:56.560084  3364 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 13:39:56.930793  3364 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 13:39:57.000202  3364 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 13:39:57.100350  3364 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0912 13:39:58.498713  3364 deprecation_wrapper.py:119] From C:\ProgramData\Ana

In [ ]:


#training data
generator_top = datagen.flow_from_directory( 
   train_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 85552 images belonging to 61 classes.


In [ ]:


#validation data\n",
generator_top = datagen.flow_from_directory(
             validation_data_dir,  
             target_size=(img_width, img_height),  
             batch_size=batch_size,  
             class_mode=None,  
             shuffle=False)  
       
nb_validation_samples = len(generator_top.filenames)  
       
validation_data = np.load('bottleneck_features_validation.npy')  
       
    
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

In [ ]:


#testing data\n",
generator_top = datagen.flow_from_directory(  
             test_data_dir,  
             target_size=(img_width, img_height),  
             batch_size=batch_size,  
             class_mode=None,  
             shuffle=False)  
       
nb_test_samples = len(generator_top.filenames)  
       
test_data = np.load('bottleneck_features_test.npy')  
       
    
test_labels = generator_top.classes 
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
start = datetime.datetime.now()
model = Sequential() 
model.add(Flatten(input_shape=train_data.shape[1:])) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.5)) 
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.RMSprop(lr=1e-4),
   metrics=['acc'])
history = model.fit(train_data, train_labels, 
   epochs=4,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
#print("[INFO] Loss: {}".format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
class ContourWithData():

    # member variables ############################################################################
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    def calculateRectTopLeftPointAndWidthAndHeight(self):               # calculate bounding rect info
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    def checkIfContourIsValid(self):                            # this is oversimplified, for a production grade program
        if self.fltArea < MIN_CONTOUR_AREA: return False        # much better validity checking would be necessary
        return True

In [ ]:
from numpy import*

#Image Resize

RPath = r'C:\Users\th4pa\93.jpg'
basewidth = 400
img = Image.open(RPath)
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize), Image.ANTIALIAS)
img.save(RPath)

#Image Segment

temp=Image.open(RPath)                      # Take Image
temp=temp.convert('1')                        # Convert to black & white
A = array(temp)                               # Creates an array, white pixels==True and black pixels==False
new_A= empty((A.shape[0],A.shape[1]),None)    # New array with same size as A

for i in range(len(A)):
    for j in range(len(A[i])):
        if A[i][j]==True:
            new_A[i][j]=255                   # Set Pixel Values 
        else:
            new_A[i][j]=0                     # Set Pixel Values
            

workbook = xlsxwriter.Workbook('test.xlsx')   # Create File
worksheet = workbook.add_worksheet()
for i in range(len(A)):
    for j in range(len(A[i])):
        worksheet.write(i, j, new_A[i][j])    # Write to File
workbook.close()


array = np.array(new_A, dtype=np.uint8)       # Convert the pixels into an array using numpy
new_image = Image.fromarray(array)            # Use PIL to create an image from the new array of pixels
new_image.save('new.png')                     # Save New Image
#new_image.show()


max_cnt = 0
target_row = 0
similar = 0
for i in range(len(A)):
    count = 0         
    for j in range(len(A[i])):
        if (new_A[i][j] == 0):
            count += 1                        # Make Temporary Count
    if (count > max_cnt):
        max_cnt = count
        target_row = i                        # Select The "Matra" Row
    if (count == max_cnt or count == max_cnt - 3 or count == max_cnt + 3):
        if (i in range(target_row - 4, target_row + 4)):
            similar += 1
#print (target_row, take)


for i in range(len(A[i])):
    count = 0
    for j in range(len(A)):
        if (new_A[j][i] == 0):
            count += 1
    if (count <= (similar)):
        for l in range(target_row - similar, target_row + 4):
            if (new_A[l][i] == 0):
                new_A[l][i] = 255
                
#Upper Zone
                
for i in range ((target_row - similar - 2), (target_row - similar + 2)):      
    for j in range(len(A[i])):
        if (new_A[i][j] == 0):
            new_A[i][j] = 255

array = np.array(new_A, dtype=np.uint8)       # Convert the pixels into an array using numpy
new_image = Image.fromarray(array)            # Use PIL to create an image from the new array of pixels
new_image.save('new1.png')                     # Save New Image
            
#Lowest Level

lower = 5000

imgTrainingNumbers = cv2.imread('new1.png')            # read in training numbers image
allContoursWithData = []                # declare empty lists,
validContoursWithData = []              # we will fill these shortly

imgTestingNumbers = cv2.imread('new1.png')              # read in testing numbers image

if imgTestingNumbers is None:                           # if image was not read successfully
    print ("error: image not read from file \n\n")        # print error message to std out
    os.system("pause")                                  # pause so user can see error message
    #return                                              # and exit function (which exits program)
# end if

imgGray = cv2.cvtColor(imgTestingNumbers, cv2.COLOR_BGR2GRAY)       # get grayscale image
imgBlurred = cv2.GaussianBlur(imgGray, (5,5), 0)                    # blur

                                                    # filter image from grayscale to black and white
imgThresh = cv2.adaptiveThreshold(imgBlurred,                           # input image
                                  255,                                  # make pixels that pass the threshold full white
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,       # use gaussian rather than mean, seems to give better results
                                  cv2.THRESH_BINARY_INV,                # invert so foreground will be white, background will be black
                                  11,                                   # size of a pixel neighborhood used to calculate threshold value
                                  2)                                    # constant subtracted from the mean or weighted mean

imgThreshCopy = imgThresh.copy()        # make a copy of the thresh image, this in necessary b/c findContours modifies the image

npaContours, npaHierarchy = cv2.findContours(imgThreshCopy,             # input image, make sure to use a copy since the function will modify this image in the course of finding contours
                                             cv2.RETR_EXTERNAL,         # retrieve the outermost contours only
                                             cv2.CHAIN_APPROX_SIMPLE)   # compress horizontal, vertical, and diagonal segments and leave only their end points

for npaContour in npaContours:                             # for each contour
    contourWithData = ContourWithData()                                             # instantiate a contour with data object
    contourWithData.npaContour = npaContour                                         # assign contour to contour with data
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)     # get the bounding rect
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()                    # get bounding rect info
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)           # calculate the contour area
    allContoursWithData.append(contourWithData)                                     # add contour with data object to list of all contours with data
# end for

for contourWithData in allContoursWithData:                 # for all contours
    if contourWithData.checkIfContourIsValid():             # check if valid
        validContoursWithData.append(contourWithData)       # if so, append to valid contour list
    # end if
# end for

validContoursWithData.sort(key = operator.attrgetter("intRectX"))         # sort contours from left to right

for contourWithData in validContoursWithData:            # for each contour
                                            # draw a green rect around the current char
    cv2.rectangle(imgTestingNumbers,                                        # draw rectangle on original testing image
                  (contourWithData.intRectX, contourWithData.intRectY),     # upper left corner
                  (contourWithData.intRectX + contourWithData.intRectWidth, contourWithData.intRectY + contourWithData.intRectHeight),      # lower right corner
                  (0, 255, 0),              # green
                  2)                        # thickness

    imgROI = imgThresh[contourWithData.intRectY : contourWithData.intRectY + contourWithData.intRectHeight,     # crop char out of threshold image
                       contourWithData.intRectX : contourWithData.intRectX + contourWithData.intRectWidth]
    
    if ((contourWithData.intRectY >= target_row - similar) and 
        (contourWithData.intRectY + contourWithData.intRectHeight < lower) and 
        (contourWithData.intRectHeight > similar * 2)):
        lower = contourWithData.intRectY + contourWithData.intRectHeight
        #print (lower)
        #print (contourWithData.intRectHeight)
    
    #cv2.imshow("imgTestingNumbers1", imgTestingNumbers)
    #cv2.imshow("imgROI", imgROI)
    #plt.imsave('test.png', imgROI)
    

    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))             # resize image, this will be more consistent for recognition and storage

    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))      # flatten image into 1d numpy array

    npaROIResized = np.float32(npaROIResized)       # convert from 1d numpy array of ints to 1d numpy array of floats

    intChar = cv2.waitKey(0)                     # get key press
# end for
#print ("\n" + strFinalString + "\n")                  # show the full string

#cv2.imshow("imgTestingNumbers1", imgTestingNumbers)      # show input image with green boxes drawn around found digits
cv2.waitKey(0)                                          # wait for user key press

cv2.destroyAllWindows()             # remove windows from memory

#Lower Zone

for i in range ((lower), (lower + similar)):      
    for j in range(len(A[i])):
        if (new_A[i][j] == 0):
            new_A[i][j] = 255
            
array = np.array(new_A, dtype=np.uint8)       # Convert the pixels into an array using numpy
new_image = Image.fromarray(array)            # Use PIL to create an image from the new array of pixels
new_image.save('new2.png')                     # Save New Image

In [ ]:
import array as arr

take = arr.array ('u', ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'])
take1 = 0
take2 = 0
take3 = 0
take4 = 0
take5 = 0
take6 = 0
take7 = 0
take8 = 0

In [ ]:
def read_image(file_path):
   #print("[INFO] loading and preprocessing image…") 
   image = load_img(file_path, target_size=(224, 224)) 
   image = img_to_array(image) 
   image = np.expand_dims(image, axis=0)
   image /= 255. 
   return image

imgTrainingNumbers = cv2.imread("new1.png")            # read in training numbers image
allContoursWithData = []                # declare empty lists,
validContoursWithData = []              # we will fill these shortly

imgTestingNumbers = cv2.imread("new1.png")              # read in testing numbers image

if imgTestingNumbers is None:                           # if image was not read successfully
    print ("error: image not read from file \n\n")        # print error message to std out
    os.system("pause")                                  # pause so user can see error message
    #return                                              # and exit function (which exits program)
# end if

imgGray = cv2.cvtColor(imgTestingNumbers, cv2.COLOR_BGR2GRAY)       # get grayscale image
imgBlurred = cv2.GaussianBlur(imgGray, (5,5), 0)                    # blur

                                                    # filter image from grayscale to black and white
imgThresh = cv2.adaptiveThreshold(imgBlurred,                           # input image
                                  255,                                  # make pixels that pass the threshold full white
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,       # use gaussian rather than mean, seems to give better results
                                  cv2.THRESH_BINARY_INV,                # invert so foreground will be white, background will be black
                                  11,                                   # size of a pixel neighborhood used to calculate threshold value
                                  2)                                    # constant subtracted from the mean or weighted mean

imgThreshCopy = imgThresh.copy()        # make a copy of the thresh image, this in necessary b/c findContours modifies the image

npaContours, npaHierarchy = cv2.findContours(imgThreshCopy,             # input image, make sure to use a copy since the function will modify this image in the course of finding contours
                                             cv2.RETR_EXTERNAL,         # retrieve the outermost contours only
                                             cv2.CHAIN_APPROX_SIMPLE)   # compress horizontal, vertical, and diagonal segments and leave only their end points

for npaContour in npaContours:                             # for each contour
    contourWithData = ContourWithData()                                             # instantiate a contour with data object
    contourWithData.npaContour = npaContour                                         # assign contour to contour with data
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)     # get the bounding rect
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()                    # get bounding rect info
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)           # calculate the contour area
    allContoursWithData.append(contourWithData)                                     # add contour with data object to list of all contours with data
# end for

for contourWithData in allContoursWithData:                 # for all contours
    if contourWithData.checkIfContourIsValid():             # check if valid
        validContoursWithData.append(contourWithData)       # if so, append to valid contour list
    # end if
# end for

validContoursWithData.sort(key = operator.attrgetter("intRectX"))         # sort contours from left to right

#strFinalString = ""         # declare final string, this will have the final number sequence by the end of the program

flag = flag1 = 0
cnt = 1

for contourWithData in validContoursWithData:            # for each contour
                                            # draw a green rect around the current char
    cv2.rectangle(imgTestingNumbers,                                        # draw rectangle on original testing image
                  (contourWithData.intRectX, contourWithData.intRectY),     # upper left corner
                  (contourWithData.intRectX + contourWithData.intRectWidth, contourWithData.intRectY + contourWithData.intRectHeight),      # lower right corner
                  (0, 255, 0),              # green
                  2)                        # thickness

    imgROI = imgThresh[contourWithData.intRectY : contourWithData.intRectY + contourWithData.intRectHeight,     # crop char out of threshold image
                       contourWithData.intRectX : contourWithData.intRectX + contourWithData.intRectWidth]
    
    if (contourWithData.intRectY >= (target_row - similar)):
        take1 = contourWithData.intRectX
        take2 = contourWithData.intRectX + contourWithData.intRectWidth
        take3 = contourWithData.intRectY
        take4 = contourWithData.intRectY + contourWithData.intRectHeight
        
    else:
        take5 = contourWithData.intRectX
        take6 = contourWithData.intRectX + contourWithData.intRectWidth
        take7 = contourWithData.intRectY
        take8 = contourWithData.intRectY + contourWithData.intRectHeight
    
    #cv2.imshow("imgTestingNumbers1", imgTestingNumbers)
    #cv2.imshow("imgROI", imgROI)
    plt.imsave('test.png', imgROI)
    
    
    def test_single_image(path):
      animals = ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 
            'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 
            'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'া', 'ে', 'ু', 'ূ', 'ৃ', '০', '১', '২', '৩', '৪', '৫', '৬', '৭',
             '৮','৯']
      images = read_image(path)
      time.sleep(.5)
    
    
      bt_prediction = vgg16.predict(images) 
      preds = model.predict_proba(bt_prediction)
      per = 0
      for idx, animal, x in zip(range(0,61), animals , preds[0]):
       if (round (x*100, 2) > per):
        per = round (x*100, 2)
        container = animal
    
       #print("ID: {}, Label: {} {}%".format(idx, animal, round(x*100,2) ))
            
    
      print('Final Decision:')
      time.sleep(.5)
    
    
      for x in range(3):
       #print('.'*(x+1))
       time.sleep(.2)
      class_predicted = model.predict_classes(bt_prediction)
      class_dictionary = generator_top.class_indices 
      inv_map = {v: k for k, v in class_dictionary.items()}
    
      if (container == 'ু'):
        container = 'ড়'
      elif (container == 'ূ'):
        container = 'ঢ়'
      elif (container == 'ৃ'):
        container = 'য়'
      elif (container == 'ড়'):
        container = 'া'
      elif (container == 'ঢ়'):
        container = 'ু'
      elif (container == 'য়'):
        container = 'ূ'
      elif (container == 'া'):
        container = 'ৃ' 
    
      global flag, flag1, take1, take2, take3, take4, take5, take6, take7, take8, cnt
      
      print("Character: {}, Label: {}, Accuracy: {}%".format(class_predicted[0],  container, per))
        
      if (take3 >= (target_row - (similar * 2))):
          if ((take2 - take1 <= (similar * 2)) and ((take4 - take3) <= (similar * 2))):
                if (take[cnt - 1] == 'ড'):
                    container = 'ড়'
                    take[cnt - 1] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 17")
                    
                elif (take[cnt - 1] == 'ঢ'):
                    container = 'ঢ়'
                    take[cnt - 1] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 18")
                    
                elif (take[cnt - 1] == 'ব'):
                    container = 'র'
                    take[cnt - 1] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 19")
                    
                elif (take[cnt - 1] == 'য'):
                    container = 'য়'
                    take[cnt - 1] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 20")
                
          elif (per >= 25): 
            if (flag == 1):
                if (container == 'া'):
                    if (take6 > (take2 + 7)):
                        container = 'ি'
                    
                    elif (take1 > (take5 + 7) and take[cnt - 1] != 'ধ'):
                        container = 'ী'
                        
                    take[cnt] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 14")
                    cnt += 1
                        
                    flag = take5 = take6 = take7 = take8 = 0
                    
                elif (container == 'হ'):
                    container = 'ই'
                    take[cnt] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 5")
                    cnt += 1
                    flag = 0
                    
                elif (container == 'ড'):
                    container = 'উ'
                    take[cnt] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 7")
                    cnt += 1
                    flag = 0
                    
                elif (container == 'ঢ'):
                    container = 'ট'
                    take[cnt] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 8")
                    cnt += 1
                    flag = 0
                    
                elif (container == 'ে'):
                    container = 'ৈ'
                    take[cnt] = container
                    print ("ACCEPTED - ", container)
                    print ("logic - 9")
                    cnt += 1
                    flag = 0
                    
                elif (take[cnt - 1] == 'ি' and flag1 == 1):
                    take[cnt] = (container)
                    take[cnt - 1], take[cnt] = take[cnt], take[cnt - 1]
                    print ("ACCEPTED - ", container)
                    print ("logic - 21")
                    cnt += 1
                    flag1 = 0
                    
                else:
                    take[cnt] = (container)
                    print ("ACCEPTED - ", container)
                    print ("logic - 13")
                    cnt += 1
                
            elif (container == 'া'):
                        
                if ((take4 - take3) > (similar * 2)):
                    
                    if (take[cnt - 1] == 'ি' and flag1 == 1):
                        take[cnt] = (container)
                        take[cnt - 1], take[cnt] = take[cnt], take[cnt - 1]
                        print ("ACCEPTED - ি")
                        print ("logic - 3")
                        cnt += 1
                        flag1 = 0
                        
                    elif (take[cnt - 1] == 'ৈ'):
                        container = 'ৌ'
                        take[cnt - 1] = container
                        print ("ACCEPTED - ", container)
                        print ("logic - 15")
                        
                    else:
                        take[cnt] = container
                        print ("ACCEPTED - ", container)
                        print ("logic - 16")
                        cnt += 1
                    
                take1 = take2 = take3 = take4 = 0
            
            else:
                take[cnt] = container
                print ("ACCEPTED - ", container)
                print ("logic - 1")
                
                if (flag1 == 1 and (take[cnt - 1] == 'ি' or take[cnt - 1] == 'ে' or take[cnt - 1] == 'ৈ')):
                    take[cnt - 1], take[cnt] = take[cnt], take[cnt - 1]
                    flag1 = 0
                    
                elif (container == 'ে'):
                    flag1 = 1
                
                cnt += 1
                
                take1 = take2 = take3 = take4 = 0
                
          else:
            take1 = take2 = take3 = take4 = 0
            
      elif (take8 <= target_row):
        if (take[cnt - 1] == 'া'):
            container = 'ি'
            take[cnt - 1] = container
            print ("ACCEPTED - ", container)
            print ("logic - 4")
            flag1 = 0
            
        if (take[cnt - 1] == 'হ'):
            container = 'ই'
            take[cnt - 1] = container
            print ("ACCEPTED - ", container)
            print ("logic - 6")
            flag1 = 1
            
        if (take[cnt - 1] == 'ড'):
            container = 'উ'
            take[cnt - 1] = container
            print ("ACCEPTED - ", container)
            print ("logic - 10")
            flag1 = 1
            
        if (take[cnt - 1] == 'ঢ'):
            container = 'ট'
            take[cnt - 1] = container
            print ("ACCEPTED - ", container)
            print ("logic - 11")
            flag1 = 1
            
        if (take[cnt - 1] == 'ে'):
            container = 'ৈ'
            take[cnt - 1] = container
            print ("ACCEPTED - ", container)
            print ("logic - 12")
            flag1 = 1
        
        else:    
            flag = 1
            flag1 = 1
    
    path = (r'C:\Users\th4pa\Bangla\test.png')
    test_single_image(path)
    

    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))             # resize image, this will be more consistent for recognition and storage

    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))      # flatten image into 1d numpy array

    npaROIResized = np.float32(npaROIResized)       # convert from 1d numpy array of ints to 1d numpy array of floats

    #intChar = cv2.waitKey(0)                     # get key press
# end for

#print ("\n" + strFinalString + "\n")                  # show the full string

#cv2.imshow("imgTestingNumbers1", imgTestingNumbers)      # show input image with green boxes drawn around found digits
cv2.waitKey(0)                                          # wait for user key press

cv2.destroyAllWindows()             # remove windows from memory

In [ ]:
word = ""
for temp in range (1, cnt):
    word = word + take[temp]
    
print (word)